In [ ]:
import boto3

elb = boto3.client('elb')
ec2 = boto3.client('ec2')

In [ ]:
my_session = boto3.session.Session()
my_region = my_session.region_name
print my_region

In [ ]:
# AMIs are unique per region. Here we will look up our AMI based on our region
amis = {}
amis['us-east-1'] = 'ami-0b33d91d'
amis['us-east-2'] = 'ami-446f3521'
amis['us-west-1'] = 'ami-9fadf8ff'
amis['us-west-2'] = 'ami-7abc111a'
amis['eu-west-1'] = 'ami-a1491ad2'
amis['ca-central-1'] = 'ami-ebed508f'

ami_id = amis[my_region]
print ami_id

In [ ]:
user_data = \
"""#!/bin/bash
yum update -y
yum install httpd -y
cd /var/www/html
echo "<html><h1>Hello cloud gurus</h1></html>" > /var/www/html/index.html
service httpd start"""

In [ ]:
# Make a web security group for the load balancer
response = ec2.create_security_group(
    GroupName='web_sg',
    Description='SG for web ingres'
)

webSGID = response['GroupId']
print 'created security group ', webSGID

# Allow ingress for ssh
response = ec2.authorize_security_group_ingress(
    GroupId=webSGID,
    IpProtocol='tcp',
    FromPort=80,
    ToPort=80,
    CidrIp='0.0.0.0/0'
)

print response

In [ ]:
# Launch instance - default VPC, default subnet, etc.
launch_response = ec2.run_instances(
    ImageId=ami_id,
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    UserData=user_data,
    SecurityGroups=[
        'web_sg'
    ],
)


instanceId = launch_response['Instances'][0]['InstanceId']
print instanceId

In [ ]:
# What are the availability zones in the region?
response = ec2.describe_availability_zones()

zones = []
for z in response['AvailabilityZones']:
    zones.append(z['ZoneName'])
    
print zones

In [ ]:
# Create the load balancer
create_elb_response = elb.create_load_balancer(
    LoadBalancerName='sample-lb',
    Listeners=[
        {
            'Protocol': 'HTTP',
            'LoadBalancerPort': 80,
            'InstanceProtocol': 'HTTP',
            'InstancePort': 80
        },
    ],
    AvailabilityZones=[
        zones[0], zones[1]
    ],
    SecurityGroups=[
        webSGID,
    ]
)

print create_elb_response

In [ ]:
response = elb.register_instances_with_load_balancer(
     LoadBalancerName='sample-lb',
    Instances=[
        {
            'InstanceId': instanceId
        },
    ]
)

print response

In [ ]:
response = elb.describe_instance_health(
    LoadBalancerName='sample-lb',
    Instances=[
        {
            'InstanceId': instanceId
        },
    ]
)
print response

In [ ]:
response = elb.describe_load_balancers(
    LoadBalancerNames=[
        'sample-lb',
    ]
)

print response

In [ ]:
# Destroy the load balancer
response = elb.delete_load_balancer(
    LoadBalancerName='sample-lb'
)

print response

In [ ]:
# Terminate the instance
response = ec2.terminate_instances(
    InstanceIds=[
        instanceId,
    ]
)
print response



In [ ]:
# Wait for the instance to terminate
waiter = ec2.get_waiter('instance_terminated')
waiter.wait(
    InstanceIds=[
        instanceId,
    ]
)
print 'instance stopped'

In [ ]:
# Delete the security group
response = ec2.delete_security_group(
    GroupId=webSGID
)

print response